# Import Module / Library

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv

# Initialize Mediapipe 

In [2]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

# Create Header Dataset

In [3]:
landmarks = ["class", "jarak_wrist", "jarak_elbow", "elbow_to_wrist_ratio"]

In [4]:
with open('distance_wrist_elbow.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Create Function Calculate Distance

In [5]:
def hitung_jarak(a, b):
    a = np.array(a)
    b = np.array(b)
    jarak = np.linalg.norm(a - b) # Menghitung jarak Euclidean antara dua titik
    return jarak

# Create Function Store Distance

In [8]:
def export_distance(results, label):
    try:
        # Ekstraksi koordinat yang diperlukan
        coords = results.pose_landmarks.landmark
        titik_13 = [coords[13].x, coords[13].y]
        titik_14 = [coords[14].x, coords[14].y]
        titik_15 = [coords[15].x, coords[15].y]
        titik_16 = [coords[16].x, coords[16].y]

        # Hitung sudut kiri dan kanan
        jarak_wrist = hitung_jarak(titik_15, titik_16)
        jarak_elbow = hitung_jarak(titik_13, titik_14)

        elbow_to_wrist_ratio = jarak_elbow / jarak_wrist

        row = [label, jarak_wrist, jarak_elbow, elbow_to_wrist_ratio]

        with open('distance_wrist_elbow.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
    except Exception as e:
        print(f"Error: {e}")
        pass

# Extract Video to Dataset

In [10]:
cap = cv2.VideoCapture("hand_wrong.mp4")

#Initiate Holistic Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # streaming the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to the desired window size
        frame = cv2.resize(frame, (640, 480))

        # recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detections
        results = pose.process(image)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        k = cv2.waitKey(1)
        export_distance(results, 0)
        if k == ord('q'):  # Press 'q' to quit
            break
        # Stream vid result
        cv2.imshow("Raw Cam Feed", image)

    cap.release()
    cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
